# Фаза 2 • Неделя 10 • Понедельник
## Обработка естественного языка
### Рекуррентные нейронные сети • RNN & LSTM

In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import re
import string
from collections import Counter
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torchutils as tu
from torchmetrics.classification import BinaryAccuracy

In [2]:
df = pd.read_csv('data/imdb.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
def data_preprocessing(text: str) -> str:
    """preprocessing string: lowercase, removing html-tags, punctuation and stopwords

    Args:
        text (str): input string for preprocessing

    Returns:
        str: preprocessed string
    """    

    text = text.lower()
    text = re.sub('<.*?>', '', text) # html tags
    text = ''.join([c for c in text if c not in string.punctuation])# Remove punctuation
    text = [word for word in text.split() if word not in stop_words] 
    text = ' '.join(text)
    return text

df['cleaned_reviews'] = df['review'].apply(data_preprocessing)

In [4]:
corpus = [word for text in df['cleaned_reviews'] for word in text.split()]
count_words = Counter(corpus)

sorted_words = count_words.most_common()


In [5]:
def get_words_by_freq(sorted_words: list, n: int = 10) -> list:
    return list(filter(lambda x: x[1] > n, sorted_words))

In [6]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [7]:
reviews_int = []
for text in df['cleaned_reviews']:

    r = [vocab_to_int[word] for word in text.split() if vocab_to_int.get(word)]
    reviews_int.append(r)

In [8]:
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [9]:
review_len = [len(x) for x in reviews_int]
df['Review len'] = review_len
df.head()

,review,sentiment,cleaned_reviews,Review len
0,One of the other reviewers has mentioned that ...,1,one reviewers mentioned watching 1 oz episode ...,168
1,A wonderful little production. <br /><br />The...,1,wonderful little production filming technique ...,84
2,I thought this was a wonderful way to spend ti...,1,thought wonderful way spend time hot summer we...,86
3,Basically there's a family where a little boy ...,0,basically theres family little boy jake thinks...,67
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love time money visually stunni...,125


In [10]:
def padding(review_int: list, seq_len: int) -> np.array:
    """Make left-sided padding for input list of tokens

    Args:
        review_int (list): input list of tokens
        seq_len (int): max length of sequence, it len(review_int[i]) > seq_len it will be trimmed, else it will be padded by zeros

    Returns:
        np.array: padded sequences
    """    
    features = np.zeros((len(reviews_int), seq_len), dtype = int)
    for i, review in enumerate(review_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            new = zeros + review
        else:
            new = review[: seq_len]
        features[i, :] = np.array(new)
            
    return features

In [11]:
def preprocess_single_string(input_string: str, seq_len: int, vocab_to_int: dict = vocab_to_int) -> list:
    """Function for all preprocessing steps on a single string

    Args:
        input_string (str): input single string for preprocessing
        seq_len (int): max length of sequence, it len(review_int[i]) > seq_len it will be trimmed, else it will be padded by zeros
        vocab_to_int (dict, optional): word corpus {'word' : int index}. Defaults to vocab_to_int.

    Returns:
        list: preprocessed string
    """    

    preprocessed_string = data_preprocessing(input_string)
    result_list = []
    for word in preprocessed_string.split():
        try: 
            result_list.append(vocab_to_int[word])
        except KeyError as e:
            print(f'{e}: not in dictionary!')
    result_padded = padding([result_list], seq_len)[0]

    return torch.tensor(result_padded)

In [12]:
SEQ_LEN = 32
features = padding(reviews_int, SEQ_LEN)
print(features[3, :])

[  572   124   136    37   327  3493  1071   124   908  4207   630   883
 13189     1  7877  1794  1248   958  3493   933   302  5835   380 83236
    19   215    70    23     2   110   993   653]


In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(features, df['sentiment'].to_numpy(), test_size=0.2, random_state=1)

In [16]:

train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid))


BATCH_SIZE = 32

train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)

In [18]:
VOCAB_SIZE = len(vocab_to_int)+1 

In [20]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

In [33]:
device='cpu'

PyTorch: [docs](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)

#### Задание 1

Создай модель со следующими параметрами и примени `get_model_summary`: 
- два RNN-слоя (это параметр в слое, а не отдельные компоненты сети)
- 32 - размер hidden state
- 16 - размер эмбеддинг-слоя 

In [ ]:
tu.get_model_summary(
    ...
)

Задай функцию потерь, оптимизаторы и реализуй обучающий цикл. 
В цикле: 
* сохраняй промежуточные веса модели, чтобы потом не терять время на новом обучении
* замеряй время обучения текущей модели для сравнения с LSTM-моделью
* зафиксируй число эпох: 7-10 (в зависимости от ресурсов компьютера)

In [ ]:
# оптимизатор, функция потерь, метрика
pass

# цикл обучения и валидации

pass

Распечатай кривые обучения и изменения метрики. 

In [ ]:
pass

Сохрани модель и историю изменения лоссов и метрик. 

In [ ]:
pass 

#### Задание 2

Создай двунаправленную двуслойную LSTM-сеть и реши задачу классификации. 

Распечатай кривые обучения обеих моделей и значения метрик во время обучения. Примерный результат должен выглядеть так (с поправкой на твою архитектуру в заголовке):

![](aux/10-01-results.png)

Отправь полученный график в канал группы в Slack!

#### Задание 2.1

Попробуй обрезать отзывы до максимальной длины с другой стороны. В функции `padding` в случае превышения отзывом порога `seq_len` происходит обрезка – остаются только первые `seq_len` слов. Возможно, что стоит обрезать сообщения с другой стороны (оставлять только «хвост» отзыва)? 

Попробуй модифицировать функцию, обучи модель и проверь, отличается ли результат. 

#### Задание 3

Реализуй функцию, которая принимает на вход текстовое сообщение и классифицирует его обученной моделью. 

In [ ]:
def predict_sentence(text: str, model: nn.Module) -> str:
    pass
    return result

❗️ Сделай push на гитхаб!

In [ ]:
# code

#### Задание 4

Создай telegram-бота, который принимает на вход пользовательское сообщение и классифицирует его. 